#### Importing libraries

In [ ]:
#Importing libraries
import pandas as pd
import numpy as np
from zipfile import ZipFile  #To unzip data
import cv2
import matplotlib.pyplot as plt

In [ ]:
#Importing library from tensorflow
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils import Sequence
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.utils import Sequence
from tensorflow.keras.applications import ResNet50
from keras.layers import Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint

#### Read train , test csv as data frame

Read csv and add a new coloumn with .jpg appended in id( unique image number)

In [ ]:
df_train = pd.read_csv('path/train_data.csv',error_bad_lines=False)
df_train['image'] = df_train.apply(lambda row: str(row['id']) + ".jpg", axis=1)

In [ ]:
df_test = pd.read_csv('path/test_data.csv',error_bad_lines=False)
df_test['image'] = df_test.apply(lambda row: str(row['id']) + ".jpg", axis=1)

#### Use Image Data genrator to read images in batch 
#### Use Augumantion & class weight 

In [ ]:
# Data augmentation with different settings for imbalanced classes
image_generator_gender = ImageDataGenerator(
    rescale=1.0 / 255.0,  # Normalize pixel values to [0, 1]
    rotation_range=15,   # Random rotation
    width_shift_range=0.1,  # Random horizontal shift
    height_shift_range=0.1,  # Random vertical shift
    shear_range=0.1,  # Shear transformation
    zoom_range=0.1,  # Random zoom
    horizontal_flip=True,  # Random horizontal flip
    fill_mode='nearest'
)
####################################
# Define classes and count of each class
class_counts_gender = df_test['gender'].value_counts().to_dict()
#####################################
# Calculate the class weights for balancing
class_weights = {class_idx: max(class_counts_gender.values()) / count for class_idx, count in class_counts_gender.items()}

#### Reading Training data

In [ ]:
# Flow training data from DataFrame with data augmentation image
training_generator_gender= image_generator_gender.flow_from_dataframe(
    dataframe=df_train,
    directory='/content/fashion_dataset/train_image',
    x_col="image",
    y_col="gender",
    target_size=(200,200),
    batch_size=32,
    class_mode="categorical",  
    shuffle=True,
    class_weights=class_weights  # class weights for balancing
)

#### Reading test data

In [ ]:
# Flow validation data from DataFrame without augmentation
validation_generator_gender = image_generator_gender.flow_from_dataframe(
    dataframe=df_test,
    directory="/content/fashion_dataset/test_image",
    x_col="image",
    y_col="gender",
    target_size=(200,200),
    batch_size=32,
    class_mode="categorical",  # Use "categorical" for one-hot encoded labels
    shuffle=False
)

#### Create a CNN model using Rasnet

In [ ]:
#  Sequential model
model_2= Sequential()

# Load ResNet50 model
model_2.add(ResNet50(include_top=False, weights="imagenet", pooling="avg"))   # kepp include_top=False

# Adding  new layers
model_2.add(Dense(64, activation="relu"))
model_2.add(BatchNormalization())
model_2.add(Dropout(0.3))

model_2.add(Dense(32, activation="relu"))
model_2.add(BatchNormalization())
model_2.add(Dropout(0.3))

model_2.add(Dense(16, activation="relu"))
model_2.add(BatchNormalization())
model_2.add(Dropout(0.3))
model_2.add(Dense(2, activation="softmax"))

#### Complie the model

In [ ]:
# adding Early Stopping and Model Checkpoint
early_stopping = EarlyStopping(monitor="val_acc", patience=2, restore_best_weights=True)
model_checkpoint = ModelCheckpoint("best_model.h5", monitor="val_acc", save_best_only=True)
optimizer_ = Adam(learning_rate=0.0000001)         # Optimizer & learning rate
# Compile the model
model_2.compile(optimizer=optimizer_, loss="categorical_crossentropy", metrics=["accuracy"])

#### Train the model

In [ ]:
# Train the model
history_sub_cat= model_2.fit(training_generator_gender,steps_per_epoch=len(training_generator_gender),epochs=20,
    validation_data=validation_generator_gender,
    validation_steps=len(validation_generator_gender),
    callbacks=[early_stopping, model_checkpoint]
)